In [11]:
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score, classification_report
import krippendorff

In [10]:
#open platform datasets:
twitter = pd.read_excel('data/Raw_data/Twitter_full-text-merged-with-comments_n300pergenre.xlsx',dtype=str)
topic_youtube = pd.read_excel('data/Raw_data/topic_youtube_comments_and_posts_ALL_strat_sample_orderbyMB_ReadyForCoding.xlsx',dtype=str)
youtube = pd.read_excel('data/Raw_data/youtube_general_subsample2_n850.xlsx',dtype=str)
comments = pd.read_csv('data/Raw_data/Data_FixedCodingMistakes_noICR_ReadyForAnalysis.csv')

In [11]:
comments.head()

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,LIBERAL_DUMMY,CONSERVATIVE_DUMMY,NAMECALLING_DUMMY,VULGAR_DUMMY,NAMECALLING_VULGAR_DUMMY,INCIVILITY_ORDINAL,INCIVILITY_DUMMY,INTOLERANCE_DUMMY,filter_$,IMPOLITENESS_DUMMY
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,0,0,0,0,0,1,1,0,1,1
2,9/9/2021 18:49:48,9/9/2021 18:51:32,213.127.110.0,1,6,1110578710648890000,372,2,4,2,...,0,0,0,0,0,0,0,0,0,0
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [12]:
comments.columns

Index(['StartDate', 'RecordedDate', 'IPAddress', 'Finished', 'Coder', 'ID',
       'Mark_ID', 'Genre', 'topiccode', 'Platform', 'Anonymity',
       'Anonymity_9_TEXT', 'codable', 'Interaction', 'Acknowledgement',
       'TopicRelevance', 'Reasoning', 'BackgroundInfo', 'ExternalEvidence',
       'ExternalEvidence_1_TEXT', 'Opinion', 'disagreement',
       'Ideologicaldirection', 'Name_calling', 'Vulgarity',
       'Attack_reputation', 'Question_Intelligenc', 'All_caps_function',
       'Sarcasm_to_criticize', 'Individual_right', 'discrimination',
       'Invoke_violence', 'Tone', 'INTERACTIVITY_DUMMY', 'RATIONALITY_DUMMY',
       'HAS_OPINION_DUMMY', 'LIBERAL_NEUTRAL_CONSERVATIVE', 'LIBERAL_DUMMY',
       'CONSERVATIVE_DUMMY', 'NAMECALLING_DUMMY', 'VULGAR_DUMMY',
       'NAMECALLING_VULGAR_DUMMY', 'INCIVILITY_ORDINAL', 'INCIVILITY_DUMMY',
       'INTOLERANCE_DUMMY', 'filter_$', 'IMPOLITENESS_DUMMY'],
      dtype='object')

In [13]:
pd.crosstab(comments['INTERACTIVITY_DUMMY'],comments['Interaction'])

Interaction,0,1
INTERACTIVITY_DUMMY,,
0,2389,467
1,2,1004


In [14]:
pd.crosstab(comments['INTERACTIVITY_DUMMY'],comments['Acknowledgement'])

Acknowledgement,0,1
INTERACTIVITY_DUMMY,,
0,2856,0
1,0,1006


In [9]:
twitter.columns

Index(['Coder', 'id_str_x', 'twitter_name', 'Mark_ID', 'platform', 'genre',
       'user (check: name and screen_name)', '  ', 'tekst (of comment)',
       'created_at', 'full_text (original by show)', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13', 'id_str_y', 'Unnamed: 15', 'Unnamed: 0',
       'Unnamed: 0.1', 'rowNumber', 'id', 'video', 'user_id', 'truncated',
       'entities', 'source', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'geo',
       'coordinates', 'place', 'contributors', 'is_quote_status',
       'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang',
       'possibly_sensitive', 'extended_entities', 'quoted_status_id',
       'quoted_status_id_str', 'quoted_status',
       'possibly_sensitive_appealable', 'withheld_in_countries_0',
       'reply_count_per_show', 'reply_count_per_tweet'],
      dtype='object')

In [8]:
#get full data:
df = pd.read_csv('data/Data_combination_test.csv')

In [3]:
#load LIWC 22 results:
liwc = pd.read_csv('outputs/automated_results/LIWC22_swear_result.csv')

In [4]:
liwc

,Unnamed: 0,ID,Mark_ID,commentText,Segment,swear,swear_normalized,simple_swear,mean_split_swear,median_split_swear,q25_split_swear,q75_split_swear,q99_split_swear
0,0,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,sad,1,0.0,-1.891441,0,0,0,0,0,0
1,1,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,That's a vicious insult!!! What did a box of r...,1,0.0,-1.891441,0,0,0,0,0,0
2,2,1.11057871064889E+018,372,@colbertlateshow The question has always been ...,1,0.0,-1.891441,0,0,0,0,0,0
3,3,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,Goya Solidar. So there are a few of us left. ...,1,0.0,-1.891441,0,0,0,0,0,0
4,4,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,hello hello \nNo-one else will hug him.,1,0.0,-1.891441,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3856,3856,1.1522194675791E+018,10000695,@FullFrontalSamB They can’t afford chemical pe...,1,0.0,-1.891441,0,0,0,0,0,0
3857,3857,1.08536229647243E+018,10007008,@AC360 @CNN @andersoncooper It's not if..... h...,1,0.0,-1.891441,0,0,0,0,0,0
3858,3858,UghFY3QJ6nmT_ngCoAEC.7-H0Z7--wxd8goqpaPs-bl,20000102,"Nah, they knew all about the cameras. I'm gue...",1,0.0,-1.891441,0,0,0,0,0,0
3859,3859,UgyWabsmmnq3zam4DgZ4AaABAg,20000418,Alexander Hamilton. Troops are waiting in the ...,1,0.0,-1.891441,0,0,0,0,0,0


In [5]:
#load train/test data
trainset: pd.DataFrame = pd.read_csv('data/train.csv')
testset: pd.DataFrame = pd.read_csv('data/test.csv')

In [6]:
#add train/test split variable to the dataset:
liwc.loc[liwc['Mark_ID'].isin(trainset['Mark_ID']), 'split'] = 'train'
liwc.loc[liwc['Mark_ID'].isin(testset['Mark_ID']), 'split'] = 'test'

In [9]:
#merge data:
df = df.merge(liwc, on='Mark_ID', how='left')

In [10]:
df.columns

Index(['StartDate', 'RecordedDate', 'IPAddress', 'Finished', 'Coder', 'ID_x',
       'Mark_ID', 'Genre', 'topiccode', 'Platform',
       ...
       'Segment', 'swear', 'swear_normalized', 'simple_swear',
       'mean_split_swear', 'median_split_swear', 'q25_split_swear',
       'q75_split_swear', 'q99_split_swear', 'split'],
      dtype='object', length=143)

In [19]:
pd.crosstab(df.mean_split_swear, df.split)

split,test
mean_split_swear,
0,743
1,30


In [20]:
#evaluete respective performance on the TRAINING set (NOTE the wording of the few shot examples in the prompt is polished by GPT4o, not exactly like codebook):
llm_human_column_pairs = [
    ("INCIVILITY_DUMMY", "simple_swear"),
    ("INCIVILITY_DUMMY", "mean_split_swear"),
    ("INCIVILITY_DUMMY", "median_split_swear"),
    ("INCIVILITY_DUMMY", "q25_split_swear"),
    ("INCIVILITY_DUMMY", "q75_split_swear"),
    ("INCIVILITY_DUMMY", "q99_split_swear"),
   
]
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted):")

for human_col, llm_col in llm_human_column_pairs:
    subset = df.loc[df.split == 'test', [human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()

 

    print("---")
    print(f"{classification_report(human, llm)}")
    

Inter-coder reliablity: classification_report(human as gold, llm as predicted):
---
              precision    recall  f1-score   support

           0       0.57      0.98      0.72       408
           1       0.89      0.19      0.32       365

    accuracy                           0.61       773
   macro avg       0.73      0.58      0.52       773
weighted avg       0.72      0.61      0.53       773

---
              precision    recall  f1-score   support

           0       0.54      0.99      0.70       408
           1       0.87      0.07      0.13       365

    accuracy                           0.56       773
   macro avg       0.71      0.53      0.42       773
weighted avg       0.70      0.56      0.43       773

---
              precision    recall  f1-score   support

           0       0.55      0.99      0.71       408
           1       0.88      0.10      0.18       365

    accuracy                           0.57       773
   macro avg       0.71      0.54   

c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
